In [13]:
import pandas as pd
from urllib.request import urlopen
import json

def date_format(d=''):    #날짜를 파이썬이 사용하는 실제 날짜 형식으로 변환
    if d!= '':
        this_date = pd.to_datetime(d).date()   
    else:
        this_date = pd.Timestamp.today().date()   #빈칸일 경우 오늘 날짜입력
    return (this_date)

def index_global(d, symbol,start_date='',end_date='',page=1):  #json 크롤링
    end_date = date_format(end_date)
    if start_date == '':
        start_date = end_date - pd.DateOffset(months=1)
    start_date = date_format(start_date)
    
    url='https://finance.naver.com/world/worldDayListJson.nhn?symbol='+symbol+'&fdtc=0&page=' +str(page)
    raw = urlopen(url)
    data = json.load(raw)
    
    if len(data)>0:        #페이지네이션
        for n in range(len(data)):
            date = pd.to_datetime(data[n]['xymd']).date()
            if date <= end_date and date >= start_date:
                price = float(data[n]['clos'])
                d[date]=price
            elif date <start_date:
                return(d)
    if len(data) == 10:
        page+=1
        index_global(d,symbol,start_date,end_date,page)
    return (d)

indices = {
    'SPI@SPX' : 'S&P 500',
    'NAS@NDX' : 'Nasdaq 100',
    'NII@NI225' : 'Nikkei 225'
}


historical_indices = dict()
start_date = '2020-07-01'
end_date = '2019-03-01'
for key, value in indices.items() :
    s = dict()
    s = index_global(s, key, start_date)
    historical_indices[value] = s
prices_df = pd.DataFrame(historical_indices)
prices_df

,S&P 500,Nasdaq 100,Nikkei 225
2020-07-09,3152.05,10754.59,22529.29
2020-07-08,3169.94,10666.70,22438.65
2020-07-07,3145.32,10524.01,22614.69
2020-07-06,3179.72,10604.06,22714.44
2020-07-02,3130.01,10341.89,22145.96
2020-07-01,3115.86,10279.25,22121.73
2020-07-03,NaN,NaN,22306.48
